<a href="https://colab.research.google.com/github/41371108H/114-1repo/blob/main/Hw5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [26]:
!pip install google-search-results folium geopy gradio -q

In [27]:
from serpapi import GoogleSearch
import folium
from geopy.geocoders import Nominatim
import json
import re
from datetime import datetime
from IPython.display import display
import gradio as gr

In [28]:
SERPAPI_KEY = "a636872ddfc824f7e7b6d2ef17503619640c7d9e4a6e76ede0ec4ec6feb22cb8"

## 4. 定義搜尋與地理位置函數

In [29]:
# 初始化地理編碼器
geolocator = Nominatim(user_agent="search_map_app")


TAIWAN_LOCATIONS = [
    # 直轄市
    '台北', '台北市', '臺北', '臺北市', 'Taipei',
    '新北', '新北市', 'New Taipei',
    '桃園', '桃園市', 'Taoyuan',
    '台中', '台中市', '臺中', '臺中市', 'Taichung',
    '台南', '台南市', '臺南', '臺南市', 'Tainan',
    '高雄', '高雄市', 'Kaohsiung',

    # 縣市
    '基隆', '基隆市', 'Keelung',
    '新竹', '新竹市', '新竹縣', 'Hsinchu',
    '苗栗', '苗栗縣', 'Miaoli',
    '彰化', '彰化縣', 'Changhua',
    '南投', '南投縣', 'Nantou',
    '雲林', '雲林縣', 'Yunlin',
    '嘉義', '嘉義市', '嘉義縣', 'Chiayi',
    '屏東', '屏東縣', 'Pingtung',
    '宜蘭', '宜蘭縣', 'Yilan',
    '花蓮', '花蓮縣', 'Hualien',
    '台東', '台東縣', '臺東', '臺東縣', 'Taitung',
    '澎湖', '澎湖縣', 'Penghu',
    '金門', '金門縣', 'Kinmen',
    '連江', '連江縣', '馬祖', 'Matsu',

    # 知名景點/地標
    '101', '台北101', '西門町', '信義區', '中正區',
    '士林', '北投', '淡水', '九份', '平溪',
    '日月潭', '阿里山', '墾丁', '太魯閣',
    '東大門', '逢甲', '一中街', '勤美',
]

def search_google(query, api_key, num=5, gl='tw', hl='zh-tw'):
    """
    使用 SerpAPI 進行 Google 搜尋
    """
    try:
        params = {
            'q': query,
            'api_key': api_key,
            'num': num,
            'gl': gl,
            'hl': hl
        }

        search = GoogleSearch(params)
        results = search.get_dict()

        return results.get('organic_results', [])

    except Exception as e:
        print(f"搜尋時發生錯誤: {e}")
        return []


def extract_locations_from_text(text):
    """
    從文字中擷取地區名稱

    參數:
        text (str): 要分析的文字

    回傳:
        list: 找到的地區名稱列表
    """
    found_locations = []
    text_lower = text.lower()

    for location in TAIWAN_LOCATIONS:
        if location.lower() in text_lower:
            # 避免重複加入相似地名
            base_name = location.replace('市', '').replace('縣', '')
            if base_name not in [loc.replace('市', '').replace('縣', '') for loc in found_locations]:
                found_locations.append(location)

    return found_locations


def get_coordinates(location_name):
    """
    取得地點的經緯度座標

    參數:
        location_name (str): 地點名稱

    回傳:
        tuple: (緯度, 經度) 或 None
    """
    try:
        # 加上台灣以提高準確度
        location = geolocator.geocode(f"{location_name}, Taiwan", timeout=10)
        if location:
            return (location.latitude, location.longitude)
        return None
    except Exception as e:
        print(f"無法取得 {location_name} 的座標: {e}")
        return None


def analyze_results_with_locations(results):
    """
    分析搜尋結果並擷取地理位置資訊

    參數:
        results (list): 搜尋結果列表

    回傳:
        list: 包含位置資訊的結果字典列表
    """
    analyzed_results = []

    for result in results:
        # 從標題和描述中尋找地點
        title = result.get('title', '')
        snippet = result.get('snippet', '')
        combined_text = f"{title} {snippet}"

        locations = extract_locations_from_text(combined_text)

        if locations:
            analyzed_results.append({
                'title': title,
                'snippet': snippet,
                'link': result.get('link', ''),
                'locations': locations
            })

    return analyzed_results


def create_map_from_results(analyzed_results, center=[23.5, 121.0], zoom=7):
    """
    根據分析結果建立地圖

    參數:
        analyzed_results (list): 包含位置資訊的結果列表
        center (list): 地圖中心座標 [緯度, 經度]
        zoom (int): 地圖縮放層級

    回傳:
        folium.Map: 地圖物件
    """
    # 建立地圖
    m = folium.Map(location=center, zoom_start=zoom, tiles='OpenStreetMap')

    # 用於記錄已標註的位置，避免重複
    marked_locations = {}

    # 為每個結果加上標記
    for result in analyzed_results:
        for location in result['locations']:
            coords = get_coordinates(location)

            if coords:
                lat, lon = coords

                # 如果該位置還沒標記過
                if location not in marked_locations:
                    marked_locations[location] = []

                # 將結果加到該位置
                marked_locations[location].append(result)

    # 在地圖上標註
    colors = ['red', 'blue', 'green', 'purple', 'orange', 'darkred',
              'lightred', 'beige', 'darkblue', 'darkgreen', 'cadetblue']

    for i, (location, results) in enumerate(marked_locations.items()):
        coords = get_coordinates(location)
        if coords:
            lat, lon = coords

            # 建立 popup 內容
            popup_html = f"<div style='width: 300px;'>"
            popup_html += f"<h4 style='color: #2c3e50;'>{location}</h4>"
            popup_html += f"<p style='color: #7f8c8d; font-size: 12px;'>找到 {len(results)} 筆相關結果</p>"

            for j, res in enumerate(results, 1):
                popup_html += f"<div style='margin: 10px 0; padding: 10px; background: #ecf0f1; border-radius: 5px;'>"
                popup_html += f"<b style='color: #2980b9;'>[{j}] {res['title'][:50]}...</b><br>"
                popup_html += f"<small style='color: #34495e;'>{res['snippet'][:100]}...</small><br>"
                popup_html += f"<a href='{res['link']}' target='_blank' style='color: #3498db;'>查看連結</a>"
                popup_html += "</div>"

            popup_html += "</div>"

            # 加上標記
            folium.Marker(
                location=[lat, lon],
                popup=folium.Popup(popup_html, max_width=350),
                tooltip=f"{location} ({len(results)} 筆結果)",
                icon=folium.Icon(color=colors[i % len(colors)], icon='info-sign')
            ).add_to(m)

    return m


def display_results_summary(analyzed_results):
    """
    顯示分析結果摘要
    """
    print("\n" + "="*80)
    print(f"找到 {len(analyzed_results)} 筆包含地理位置的搜尋結果")
    print("="*80 + "\n")

    for i, result in enumerate(analyzed_results, 1):
        print(f"【結果 {i}】")
        print(f"標題: {result['title']}")
        print(f"地點: {', '.join(result['locations'])}")
        print(f"描述: {result['snippet'][:100]}...")
        print(f"網址: {result['link']}")
        print("-"*80 + "\n")

## 5.搜尋並標註地圖

In [30]:
def search_and_map(query, api_key, num_results=10):
    """
    整合搜尋與地圖標註的主函數

    參數:
        query (str): 搜尋關鍵字
        api_key (str): SerpAPI 金鑰
        num_results (int): 搜尋結果數量

    回傳:
        folium.Map: 標註好的地圖
    """
    print(f"正在搜尋: {query}\n")

    # 1. 執行搜尋
    results = search_google(query, api_key, num=num_results)

    if not results:
        return "沒有找到任何結果"

    print(f"✓ 找到 {len(results)} 筆搜尋結果")

    # 2. 分析結果並擷取地理位置
    analyzed_results = analyze_results_with_locations(results)

    if not analyzed_results:
        return "\n⚠️ 搜尋結果中沒有找到可辨識的台灣地區"

    print(f"✓ 從結果中找到 {len(analyzed_results)} 筆包含地理位置的資料\n")

    # 3. 顯示結果摘要
    display_results_summary(analyzed_results)

    # 4. 建立地圖
    print("正在建立地圖...\n")
    map_obj = create_map_from_results(analyzed_results)
    print("✓ 地圖建立完成！")

    return map_obj

## 6. 使用範例

In [31]:
#搜尋台灣美食
query = "台灣必吃美食推薦"
map_result = search_and_map(query, SERPAPI_KEY, num_results=10)

if map_result:
    display(map_result)

正在搜尋: 台灣必吃美食推薦

✓ 找到 9 筆搜尋結果
✓ 從結果中找到 2 筆包含地理位置的資料


找到 2 筆包含地理位置的搜尋結果

【結果 1】
標題: 【台灣美食】從南吃到北！全台必吃美食大集合
地點: 台北, 基隆
描述: 台灣美食 · 基隆美食不只廟口？！必吃排骨飯、魷魚羹、龍蝦麻糬別錯過 基隆是許多台北人週末出門的好地方，不管是想欣賞海景還是大啖美食，基隆通通都能滿足。 · 【基隆名產】在 ......
網址: https://www.funtime.com.tw/theme/taiwan-food
--------------------------------------------------------------------------------

【結果 2】
標題: 在地人推薦的20+種道地台灣特色美食
地點: 台南, 高雄, 彰化
描述: 台灣各縣市必吃清單？台灣地方特色美食 ; 彰化. 肉圓、爌肉飯、麻糬 ; 台南. 牛肉湯、擔仔麵、棺材板、魚粥、安平豆花 ; 高雄. 木瓜牛奶、海產粥、鹽埕埔鴨肉 ......
網址: https://www.tripool.app/zh-HK/articles/taiwan-food
--------------------------------------------------------------------------------

正在建立地圖...

✓ 地圖建立完成！


In [32]:
#搜尋台灣景點
query = "台灣熱門觀光景點"
map_result = search_and_map(query, SERPAPI_KEY, num_results=10)

if map_result:
    display(map_result)

正在搜尋: 台灣熱門觀光景點

✓ 找到 10 筆搜尋結果
✓ 從結果中找到 8 筆包含地理位置的資料


找到 8 筆包含地理位置的搜尋結果

【結果 1】
標題: 觀光景點
地點: 宜蘭, 日月潭
描述: 北起的東北角、北海岸及宜蘭地區，擁有各類特殊海岸地形，沿途山青水秀，天藍海深；往中南部地區，日月潭的湖光山色、八卦山賞鷹、水果之鄉的梨山、佛教聖地的獅頭山等，都可充分 ......
網址: https://www.taiwan.net.tw/m1.aspx?sNo=0001016
--------------------------------------------------------------------------------

【結果 2】
標題: 台灣熱門旅遊景點
地點: 高雄, 彰化, 南投
描述: 「橫山賞鷹平台」位於南投市永興里與彰化縣社頭鄉交界處，地勢高，可眺望彰化平原... 七坑溫泉位於高雄市桃源區，想要來此體驗泡湯樂趣的遊客，需由寶來溫泉區依指標行......
網址: https://okgo.tw/buty/
--------------------------------------------------------------------------------

【結果 3】
標題: 2025台灣景點一日遊懶人包，國內旅遊
地點: 桃園
描述: 2025推薦超過60個「桃園景點」推薦懶人包，從觀光工廠玩到森林瀑布一日遊，桃園市除了到拉拉山玩之外，千萬千萬不要忘了安排桃園景點一日遊，直接整理推薦超過 ......
網址: https://bunnyann.tw/attractions/
--------------------------------------------------------------------------------

【結果 4】
標題: 2025 年台灣絕美秘境：嚴選40 個從北到南的必去推薦景點
地點: 台北, 北投
描述: 推薦【台北必去秘境】圓山大飯店密道 · 推薦【台北內湖必去秘境】大溝溪生態治水園區 · 推薦【台北公館必去秘境】台電加羅林魚木 · 推薦【台北北投必去秘境】 ......
網址: https://hshsharehouse.com/taiwan-hidden-gems-

In [33]:
#自訂搜尋
custom_query = input("請輸入搜尋關鍵字 (包含地點相關): ")
num_results = int(input("幾筆結果? (5-20): "))

map_result = search_and_map(custom_query, SERPAPI_KEY, num_results=num_results)

if map_result:
    display(map_result)

請輸入搜尋關鍵字 (包含地點相關): 天母
幾筆結果? (5-20): 8
正在搜尋: 天母

✓ 找到 8 筆搜尋結果
✓ 從結果中找到 5 筆包含地理位置的資料


找到 5 筆包含地理位置的搜尋結果

【結果 1】
標題: 天母- 維基百科，自由的百科全書
地點: 臺北, 士林, 北投
描述: 天母是位於臺灣臺北市士林區境內的一個地區。北、東緊鄰陽明山國家公園，西以磺溪與北投區的石牌為界，南接蘭雅、芝山岩地區以外雙溪與士林福德洋、林子口相隔。...
網址: https://zh.wikipedia.org/zh-tw/%E5%A4%A9%E6%AF%8D
--------------------------------------------------------------------------------

【結果 2】
標題: 天母商圈
地點: 台北, 士林
描述: 市集地點:台北市士林區天母西路1號 #報名資訊請點 ➡ https://goo.gl/pTtD2s (每週一晚上九點開始報名當週五、六、日活動) #天母商圈 #天母市集 #創意市集 #跳蚤市場 #二手...
網址: https://www.facebook.com/lovetianmu/?locale=zh_TW
--------------------------------------------------------------------------------

【結果 3】
標題: 天母商圈發展協會: 我愛天母
地點: 臺北
描述: 天母商圈位在中山北路六、七段、天母東、西、北路與忠誠路附近，以天母廣場為中心向四周延伸，是臺北市腹地最大、也是綠地最多的商圈；這裡特色店家林立，巷道幽靜， ......
網址: https://www.tianmu.org.tw/
--------------------------------------------------------------------------------

【結果 4】
標題: 台北天母店，線上訂候位服務
地點: 台北, 101
描述: 暫未開放線上候位 · 1010湘 · 丸亀製麵 · 小紅花鐵板燒 · 牛魔王 · 瓦城 · 老乾杯 · 肉多多火鍋 · 長田本庄軒. 台北天母店 ......
網址: htt

## 7. 儲存地圖(供參考，詳細請照作業需求)


In [34]:
# 將地圖儲存為 HTML 檔案
# 移除此處的display(map_result)避免在Gradio介面重複顯示
if map_result:
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"search_map_{timestamp}.html"
    map_result.save(filename)
    print(f"\n✓ 地圖已儲存為: {filename}")
    print("可以用瀏覽器開啟檢視！")


✓ 地圖已儲存為: search_map_20251113_042621.html
可以用瀏覽器開啟檢視！


In [35]:
def gradio_search_and_map(query, num_results):
    """
    用於 Gradio 介面的包裝函數
    """
    global SERPAPI_KEY
    print(f"Gradio 收到查詢: {query}, 結果數量: {num_results}")

    # 呼叫現有的搜尋和地圖函數
    map_obj_or_message = search_and_map(query, SERPAPI_KEY, num_results=num_results)

    # 檢查返回類型，如果是字串代表是錯誤訊息
    if isinstance(map_obj_or_message, str):
        return f"<div style='color: red; font-weight: bold;'>{map_obj_or_message}</div>"
    elif map_obj_or_message:
        # 將 Folium 地圖物件轉換為 HTML 字串
        map_html = map_obj_or_message._repr_html_()
        return map_html
    else:
        return "<div style='color: red; font-weight: bold;'>未知錯誤或沒有地圖結果。</div>"

# 建立 Gradio 介面
iface = gr.Interface(
    fn=gradio_search_and_map,
    inputs=[
        gr.Textbox(label="請輸入搜尋關鍵字 (包含地點相關)", placeholder="例如：台灣特色小吃", value="台灣熱門觀光景點"),
        gr.Slider(minimum=5, maximum=20, step=1, value=10, label="搜尋結果數量")
    ],
    outputs=gr.HTML(label="地圖結果"),
    title="台灣熱門觀光景點大搜查",
    description="輸入關鍵字，我將搜尋相關資訊並標註在台灣地圖上。",
    allow_flagging="never" # 禁用 Gradio 的 flagging 功能
)

# 啟動 Gradio 介面
iface.launch()

/usr/local/lib/python3.12/dist-packages/gradio/interface.py:415: UserWarning: The `allow_flagging` parameter in `Interface` is deprecated. Use `flagging_mode` instead.
  warnings.warn(


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://6212b67e24eda3df8d.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
